In [1]:
import pandas as pd
import pickle

# Data Cleaning

In [2]:
# Import CSV – 4.1M rows of request from 2008 - 2020
df = pd.read_csv('../data/sf_311_raw.csv')

# Sort dataframe by CaseID (newest first)
df.sort_values(by='CaseID', ascending=False, inplace=True)

# Reset the index
df.reset_index(drop=True, inplace=True)

df.shape

(4108469, 47)

In [3]:
# Drop all computed columns (unnecessary)
df = df.iloc[:, 0:20]
# df = df.loc[:, ~df.columns.str.contains('^:@')]

# Drop 'Updated' column
df.drop(columns='Updated', inplace=True)

print(df.shape)
df.head()

(4108469, 19)


,CaseID,Opened,Closed,Status,Status Notes,Responsible Agency,Category,Request Type,Request Details,Address,Street,Supervisor District,Neighborhood,Police District,Latitude,Longitude,Point,Source,Media URL
0,12217125,03/15/2020 09:48:00 AM,03/15/2020 09:55:27 AM,Closed,Case is a Duplicate - This issue has already b...,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,White - -,"79 REGENT ST, SAN FRANCISCO, CA, 94112",REGENT ST,11.0,Outer Mission,INGLESIDE,37.709772,-122.456850,"(37.70977185, -122.45684959)",Phone,http://mobile311.sfgov.org/reports/12217125/ph...
1,12217120,03/15/2020 09:46:48 AM,03/15/2020 11:34:19 AM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,"1821 LARKIN ST, SAN FRANCISCO, CA, 94109",LARKIN ST,3.0,Polk Gulch,CENTRAL,37.794470,-122.419841,"(37.79447016, -122.41984125)",Mobile/Open311,NaN
2,12217118,03/15/2020 09:45:41 AM,03/15/2020 10:37:53 AM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Furniture,"1504 JACKSON ST, SAN FRANCISCO, CA, 94109",JACKSON ST,3.0,Polk Gulch,NORTHERN,37.794438,-122.419890,"(37.7944381, -122.41988987)",Mobile/Open311,NaN
3,12217114,03/15/2020 09:44:00 AM,03/15/2020 10:57:06 AM,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Only,Dark grey - Toyota Highlander - 6XUP467,"520 28TH AVE, SAN FRANCISCO, CA, 94121",28TH AVE,1.0,Outer Richmond,RICHMOND,37.779429,-122.487687,"(37.7794291, -122.4876867)",Web,NaN
4,12217108,03/15/2020 09:41:00 AM,03/15/2020 11:07:40 AM,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Electronics,Intersection of WOOLSEY ST and GOETTINGEN ST,WOOLSEY ST,9.0,Portola,BAYVIEW,37.724461,-122.405571,"(37.7244606, -122.40557098)",Phone,NaN


In [4]:
# Strip column names and format
df.columns = df.columns.str.strip().str.lower().str.replace(
    ' ', '_').str.replace('(', '').str.replace(')', '')

# Rename 'caseid' column
df.rename({'caseid': 'case_id'}, axis='columns', inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4108469 entries, 0 to 4108468
Data columns (total 19 columns):
 #   Column               Dtype  
---  ------               -----  
 0   case_id              int64  
 1   opened               object 
 2   closed               object 
 3   status               object 
 4   status_notes         object 
 5   responsible_agency   object 
 6   category             object 
 7   request_type         object 
 8   request_details      object 
 9   address              object 
 10  street               object 
 11  supervisor_district  float64
 12  neighborhood         object 
 13  police_district      object 
 14  latitude             float64
 15  longitude            float64
 16  point                object 
 17  source               object 
 18  media_url            object 
dtypes: float64(3), int64(1), object(15)
memory usage: 595.6+ MB


In [ ]:
# Convert time columns to datetime
df['opened'] = pd.to_datetime(df['opened'])
df['closed'] = pd.to_datetime(df['closed'])
# df.iloc[:, 1:4] = pd.to_datetime(df.iloc[:, 1:4].stack()).unstack()

df.shape

In [7]:
# Convert categorical columns to 'category' type
df['supervisor_district'] = pd.to_numeric(df['supervisor_district'], downcast='integer').astype('category')

cols = df[['category', 'neighborhood', 'police_district', 'source']]
for col in cols:
    df[col] = df[col].astype('category')
    
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4108469 entries, 0 to 4108468
Data columns (total 19 columns):
 #   Column               Dtype         
---  ------               -----         
 0   case_id              int64         
 1   opened               datetime64[ns]
 2   closed               datetime64[ns]
 3   status               object        
 4   status_notes         object        
 5   responsible_agency   object        
 6   category             category      
 7   request_type         object        
 8   request_details      object        
 9   address              object        
 10  street               object        
 11  supervisor_district  category      
 12  neighborhood         category      
 13  police_district      category      
 14  latitude             float64       
 15  longitude            float64       
 16  point                object        
 17  source               category      
 18  media_url            object        
dtypes: category(5), datet

In [8]:
# Pickle dataframe
df.to_pickle('../data/df_columns_to_datetime.pkl')

# Load dataframe
df = pd.read_pickle('../data/df_columns_to_datetime.pkl')

In [9]:
# Remove Lat/Long Outliers
df = df.loc[(df['latitude']<37.84) & (df['latitude']>37.70) & (df['longitude']<-122.36) & (df['longitude']>-122.51)]

In [10]:
# Create 'has_media' column 
df['has_media'] = df['media_url'].notnull().astype('int8').astype('category')

# Drop 'media_url' as we now have 'has_media'
df.drop(columns=['media_url'], inplace=True)

# Drop all rows with any missing value
df.dropna(how='any', inplace=True)

# Target Variable

SF311 appears to have a problem with requests that are not resolved since there isn't enough information or worse, the issue is not found once a team is on site. 

In [11]:
# Defining "wasteful" requests
duplicates = ['dup', 'dupe', 'duplicate', 'same', 'already', 'abated']

invalid = ['cancelled', 'case is invalid', 'no action required',
           'insufficient information provided', 'not accepted']

waste = ['not found', 'nothing found', 'none', 'gone on arrival', 'unable to locate', 'no longer',
         'no condition', 'no action', 'no response', 'no construction']

In [12]:
# Create target variable – wasteful requests
def create_target(df):
    """Return 'target' column"""
    regex_str = '''dup|same|already|abated|invalid|cancelled|insufficient|unable|gone|no\b|none\b|not\b|nothing'''
    df.insert(loc=1, column='target',
               value=df['status_notes'].str.lower().str.contains(regex_str, regex=True).astype('int8'))
    return df
    
create_target(df)

df.head()

,case_id,target,opened,closed,status,status_notes,responsible_agency,category,request_type,request_details,address,street,supervisor_district,neighborhood,police_district,latitude,longitude,point,source,has_media
0,12217125,1,2020-03-15 09:48:00,2020-03-15 09:55:27,Closed,Case is a Duplicate - This issue has already b...,Parking Enforcement Dispatch Queue,Parking Enforcement,Parking_on_Sidewalk,White - -,"79 REGENT ST, SAN FRANCISCO, CA, 94112",REGENT ST,11.0,Outer Mission,INGLESIDE,37.709772,-122.456850,"(37.70977185, -122.45684959)",Phone,1
1,12217120,0,2020-03-15 09:46:48,2020-03-15 11:34:19,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Boxed or Bagged Items,"1821 LARKIN ST, SAN FRANCISCO, CA, 94109",LARKIN ST,3.0,Polk Gulch,CENTRAL,37.794470,-122.419841,"(37.79447016, -122.41984125)",Mobile/Open311,0
2,12217118,0,2020-03-15 09:45:41,2020-03-15 10:37:53,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Furniture,"1504 JACKSON ST, SAN FRANCISCO, CA, 94109",JACKSON ST,3.0,Polk Gulch,NORTHERN,37.794438,-122.419890,"(37.7944381, -122.41988987)",Mobile/Open311,0
3,12217114,0,2020-03-15 09:44:00,2020-03-15 10:57:06,Closed,Case Resolved - Officer responded to request u...,Parking Enforcement Dispatch Queue,Parking Enforcement,Blocking_Driveway_Cite_Only,Dark grey - Toyota Highlander - 6XUP467,"520 28TH AVE, SAN FRANCISCO, CA, 94121",28TH AVE,1.0,Outer Richmond,RICHMOND,37.779429,-122.487687,"(37.7794291, -122.4876867)",Web,0
4,12217108,0,2020-03-15 09:41:00,2020-03-15 11:07:40,Closed,Case Resolved - Pickup completed.,Recology_Abandoned,Street and Sidewalk Cleaning,Bulky Items,Electronics,Intersection of WOOLSEY ST and GOETTINGEN ST,WOOLSEY ST,9.0,Portola,BAYVIEW,37.724461,-122.405571,"(37.7244606, -122.40557098)",Phone,0


In [13]:
# Reduce scope
df = df.loc[(df['opened'] >= '2018-01-01') & (df['closed'] <= '2018-12-31')]

df.shape

(529820, 20)

In [14]:
# Target variable
target_count = df['target'].value_counts()

# Print class balance
print(f'Class 0: {target_count[0]}')
print(f'Class 1: {target_count[1]}')
print(f'Proportion: {round(target_count[0] / target_count[1], 2)} : 1')
print(f'Percentage of Majority Class: {round(target_count[0] / sum(target_count), 3)*100}')

Class 0: 418298
Class 1: 111522
Proportion: 3.75 : 1
Percentage of Majority Class: 79.0


In [15]:
# Pickle
df.to_pickle('../data/01_df_clean_2018.pkl')

# SODA API (Alternative to CSV)
If we needed data to be updated frequently, the SODA API would be preferred to a CSV download.

In [ ]:
# from sodapy import Socrata
# # Unauthenticated client only works with public data sets. Note 'None'
# # in place of application token, and no username or password:
# client = Socrata('data.sfgov.org', None)

# # Authenticate the client
# domain = 'data.sfgov.org'
# app_token = '****************'
# username  = 'corraljrmiguel@gmail.com'
# password  = '****************'

# client = Socrata(domain,
#                  app_token,
#                  username=username,
#                  password=password)

# # Returns as JSON from API
# # converted to Python list of dictionaries by sodapy.
# results = client.get('vw6y-z8j6', limit=10000)
# # results = client.get('vw6y-z8j6', where='service_request_id=12167455')

# # Convert to pandas dataframe
# df = pd.DataFrame.from_records(results)